In [ ]:
import keras
import tensorflow as tf
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout

In [ ]:
# Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Пути к наборам данных
train_dir = '/content/drive/MyDrive/Keras/train'
test_dir = '/content/drive/MyDrive/Keras/test' 
img_width, img_height = 500, 500
# Размер выборки
batch_size = 32
nb_train_samples = 1034
nb_test_samples = 133


<built-in method format of str object at 0x7fd2e8cc1e70>
<built-in method format of str object at 0x7fd2e8cc1ed0>


In [ ]:
# Создание генератора изображений
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 1034 images belonging to 3 classes.
Found 128 images belonging to 3 classes.


In [ ]:
# Создание экземпляра модели сети MobileNet
Mobile_Net = MobileNet(
    weights='imagenet',
    include_top=False,
    input_shape=(img_width, img_height, 3)
)

# include_top : следует ли включать полностью подключенный уровень в верхнюю часть сети.
# weights : «imagenet» (предварительное обучение в ImageNet) или путь к загружаемому файлу весов.   

# Сверточная часть сети
Mobile_Net.trainable = True
trainable = False
for layer in Mobile_Net.layers:
  if layer.name == 'conv_dw_13':
    trainable = True
  layer.trainable = trainable

Mobile_Net.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 500, 500, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 250, 250, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 250, 250, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 250, 250, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 250, 250, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 250, 250, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 250, 250, 32

In [ ]:
# Создание модели составной сети
model = Sequential()
# Добавляем сверточные слои
model.add(Mobile_Net)
# Преобразуем двумерный массив MobileNet в одномерный
model.add(Flatten())
# Полносвязный слой
model.add(Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(Dropout(0.5))
# Кол-во классов
model.add(Dense(3, activation='sigmoid'))
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Function (None, 15, 15, 1024)      3228864   
_________________________________________________________________
flatten_13 (Flatten)         (None, 230400)            0         
_________________________________________________________________
dense_26 (Dense)             (None, 256)               58982656  
_________________________________________________________________
dropout_13 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 3)                 771       
Total params: 62,212,291
Trainable params: 60,045,315
Non-trainable params: 2,166,976
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd', 
              metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
32/32 [==============================] - 18s 443ms/step - loss: 12.2961 - accuracy: 0.4837 - val_loss: 0.6538 - val_accuracy: 0.6406
Epoch 2/10
32/32 [==============================] - 12s 392ms/step - loss: 0.9419 - accuracy: 0.6122 - val_loss: 0.5435 - val_accuracy: 0.8125
Epoch 3/10
32/32 [==============================] - 12s 394ms/step - loss: 0.7603 - accuracy: 0.6871 - val_loss: 0.5528 - val_accuracy: 0.7422
Epoch 4/10
32/32 [==============================] - 12s 387ms/step - loss: 0.6305 - accuracy: 0.7253 - val_loss: 0.4330 - val_accuracy: 0.8594
Epoch 5/10
32/32 [==============================] - 12s 388ms/step - loss: 0.4621 - accuracy: 0.8237 - val_loss: 0.2303 - val_accuracy: 0.9375
Epoch 6/10
32/32 [==============================] - 12s 388ms/step - loss: 0.3329 - accuracy: 0.8755 - val_loss: 0.2548 - val_accuracy: 0.8906
Epoch 7/10
32/32 [==============================] - 12s 384ms/step - loss: 0.3406 - accuracy: 0.8794 - val_loss: 0.1898 - val_accuracy: 0.945

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print('Точность работы на тестовых данных: %.2f%%' % (scores[1]*100))

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1948: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Точность работы на тестовых данных: 95.31%
